___
# Ciência dos Dados - PROJETO 2

___
## Giovanni dos Santos

## Gustavo Tessitore

## Victor Niubó

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [44]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import os.path
import numpy as np
import math
from random import shuffle

In [45]:
emails = pd.read_excel("spamham2019.xlsx") #lê o excel
remover = [',','@','/',':',';','r$','\n','\(','\)','#','\?','-','"','\[','\]']

for i in remover:
    emails.Email = emails.Email.replace(i,'',regex=True)

In [54]:
dicio = emails.to_dict('records')
shuffle(dicio)
df2 = pd.DataFrame.from_dict(dicio)
df3 = df2.head(int(len(df2)*0.75))
df4 = df2.tail(int(len(df2)*0.05))

___
## 3. Classificador Naive-Bayes

In [47]:
spam = df3.Class

prob = spam.value_counts()

prob_ham = prob[0]/len(spam) 

prob_spam = prob[1]/len(spam) 

print("Probabilidade de ser relevante:",int(prob_ham * 100),'%')
print('Probabilidade de ser irrelevante',int(prob_spam * 100),'%')

Probabilidade de ser relevante: 86 %
Probabilidade de ser irrelevante 13 %


In [48]:
presunto = df3[df3.Class == 'ham'] #Separa os presuntos relevantes
relevantes = np.sum(presunto.Email).split()
quant_em_relev =  pd.Series(relevantes)
todos_relevantes = len(relevantes)
quant_em_relev.value_counts().head(3)

to     1132
you    1124
I       815
dtype: int64

In [49]:
queijo = df3[df3.Class == 'spam'] #Separa os queijos relevantes
irelevantes = np.sum(queijo.Email).split()
quant_em_irelev =  pd.Series(irelevantes)
todos_irelevantes = len(irelevantes)
quant_em_irelev.value_counts().head(3)

to    450
a     274
or    137
dtype: int64

In [55]:
todas_as_palavras = irelevantes + relevantes



def P_spam_palavra(palavra):
    if palavra in irelevantes:
        return ((1 + quant_em_irelev.value_counts()[palavra])/(len(todas_as_palavras) + len(irelevantes)))
    else:
        return 1/(len(todas_as_palavras) + len(irelevantes))

def P_palavra_spam(palavra):
    x = (P_spam_palavra(palavra) * P_palavra(palavra)) / prob_spam
    return x

def P_palavra(palavra):
    quant_pal = (quant_em_irelev.value_counts()[palavra] + quant_em_relev.value_counts()[palavra])
    quant_tot = len(irelevantes)+len(relevantes)
    return (quant_pal/quant_tot)

def P_ham_palavra(palavra):
    if palavra in relevantes:
        return ((1 + quant_em_relev.value_counts()[palavra])/(len(todas_as_palavras) + len(relevantes)))
    else:
        return 1/(len(todas_as_palavras) + len(irelevantes))

def P_palavra_ham(palavra):
    x = (P_ham_palavra(palavra) * P_palavra(palavra)) / prob_ham
    return x

def P_spam_mensagem(mensagem):
    palavras = mensagem.split(' ')
    prob = 0
    contador = 0
    for palavra in palavras:
        if palavra in todas_as_palavras:
            if contador == 0:
                prob += P_spam_palavra(palavra)
            else:
                prob *= P_spam_palavra(palavra)
        else:
            pass
    return prob

def P_ham_mensagem(mensagem):
    palavras = mensagem.split(' ')
    prob = 0
    contador = 0
    for palavra in palavras:
        if palavra in todas_as_palavras:
            if contador == 0:
                prob += P_ham_palavra(palavra)
            else:
                prob *= P_ham_palavra(palavra)
        else:
            pass
    return prob

def Hamlet(mensagem):
    ham = P_ham_mensagem(mensagem)
    spam = P_spam_mensagem(mensagem)
    if ham > spam:
        return 'ham'
    else:
        return 'spam'
#test = []

#for i in df3.Email:
#    test.append(Hamlet(i))
#df3['NB'] = test

O teste estava demorando em torno de vinte minutos para rodar, por isso seria inviável rodá-lo 10.000 vezes. 
Portanto, segue abaixo o código do final do Projeto rodado todas as vezes.

In [87]:
test1 = []
for e in df4.Email:
    test1.append(Hamlet(e))
df4['NB'] = test1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [107]:
df5 = df4.reset_index(drop=True)

In [105]:
falso_neg = 0
falso_pos = 0
pos_ver = 0
neg_ver = 0
Spam = 0
Ham = 0

for e in range(len(df5)):
    if df5.Class[e] == 'spam':
        Spam += 1
        if df5.NB[e] == 'spam':
            pos_ver += 1
        
        elif df5.NB[e] != 'spam':
            falso_neg += 1
    
    elif df5.Class[e] == 'ham':
        Ham += 1
        if df5.NB[e] == 'ham' :
            neg_ver += 1
        
        elif df5.NB[e] != 'ham':
            falso_pos += 1
            
total = falso_neg + falso_pos + pos_ver + neg_ver            

p1= float(falso_pos/Ham * 100)
p2= float(pos_ver/Spam * 100)
p3= float(falso_neg/Spam * 100)
p4= float(neg_ver/Ham * 100)


print('Porcentagem de positivos verdadeiros: {0:.2f}%'.format(p2))

print('Porcentagem de negativos verdadeiros: {0:.2f}%'.format(p4))


Porcentagem de positivos verdadeiros: 13.20%
Porcentagem de negativos verdadeiros: 98.83%


___
## 4. Qualidade do Classificador alterando a base de treinamento

In [108]:
resultados = []

for o in range(10):
    dicio = emails.to_dict('records')
    shuffle(dicio)
    df6 = pd.DataFrame.from_dict(dicio)
    df7 = df6.head(int(len(df2)*0.75))
    df8 = df7.tail(int(len(df2)*0.25))
    df9 = df8.reset_index(drop=True)
    
    test2 = []
    for e in df9.Email:
        test1.append(Hamlet(e))
    df9['NB'] = test2
    
    posi_ver = 0
    nega_ver = 0
    Spam1 = 0
    Ham1 = 0

    for e in range(len(df9)):
        if df9.Class[e] == 'spam':
            Spam += 1
            if df9.NB[e] == 'spam':
                pos_ver += 1

            elif df9.NB[e] != 'spam':
                falso_neg += 1

        elif df9.Class[e] == 'ham':
            Ham += 1
            if df9.NB[e] == 'ham' :
                neg_ver += 1

            elif df9.NB[e] != 'ham':
                falso_pos += 1

    total = falso_neg + falso_pos + pos_ver + neg_ver            

    positivo_ver= float(pos_ver/Spam * 100)
    negativo_ver= float(neg_ver/Ham * 100)
    reults = [positivo_ver, negativo_ver]
    
    resultados.append(results)

ValueError: Length of values does not match length of index